In [1]:
!scala -version

Scala code runner version 2.11.12 -- Copyright 2002-2017, LAMP/EPFL


Setup PySpark session

In [2]:
#config is correct for Scala 2.11, not 2.12
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('BigQuery Storage & Spark DataFrames') \
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.11:0.15.1-beta') \
    .getOrCreate()

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

Access the Google Cloud Storage

In [4]:
from google.cloud import storage

gcs_client = storage.Client()
bucket = gcs_client.bucket('geitenemmer')

In [5]:
!hdfs dfs -ls "gs://geitenemmer"

Found 5 items
drwx------   - root root          0 2022-12-06 15:51 gs://geitenemmer/.ipynb_checkpoints
drwx------   - root root          0 2022-12-06 16:00 gs://geitenemmer/checkpoints
-rwx------   3 root root    6675312 2022-12-06 15:54 gs://geitenemmer/e-shop clothing 2008.csv
drwx------   - root root          0 2022-12-06 15:51 gs://geitenemmer/google-cloud-dataproc-metainfo
drwx------   - root root          0 2022-12-06 15:47 gs://geitenemmer/notebooks


Read the data in correctly

In [6]:
#read the data in

df1 = spark \
    .read \
    .option ( "inferSchema" , "true" ) \
    .option ( "header" , "true" ) \
    .option ("delimiter", ";") \
    .csv ( "gs://geitenemmer/e-shop clothing 2008.csv" )

df1.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- order: integer (nullable = true)
 |-- country: integer (nullable = true)
 |-- session ID: integer (nullable = true)
 |-- page 1 (main category): integer (nullable = true)
 |-- page 2 (clothing model): string (nullable = true)
 |-- colour: integer (nullable = true)
 |-- location: integer (nullable = true)
 |-- model photography: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- price 2: integer (nullable = true)
 |-- page: integer (nullable = true)



In [7]:
#rename columns that would have a space in the name
df1 = df1.withColumnRenamed("session ID","session_ID")
df1 = df1.withColumnRenamed("page 1 (main category)","page1_main_category")
df1 = df1.withColumnRenamed("page 2 (clothing model)","page2_clothing_model")
df1 = df1.withColumnRenamed("model photography","model_photography")
df1 =df1.withColumnRenamed("price 2","price2")

df1.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- order: integer (nullable = true)
 |-- country: integer (nullable = true)
 |-- session_ID: integer (nullable = true)
 |-- page1_main_category: integer (nullable = true)
 |-- page2_clothing_model: string (nullable = true)
 |-- colour: integer (nullable = true)
 |-- location: integer (nullable = true)
 |-- model_photography: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- price2: integer (nullable = true)
 |-- page: integer (nullable = true)



From here on the Spark analysis continues

In [8]:
df1.show()

+----+-----+---+-----+-------+----------+-------------------+--------------------+------+--------+-----------------+-----+------+----+
|year|month|day|order|country|session_ID|page1_main_category|page2_clothing_model|colour|location|model_photography|price|price2|page|
+----+-----+---+-----+-------+----------+-------------------+--------------------+------+--------+-----------------+-----+------+----+
|2008|    4|  1|    1|     29|         1|                  1|                 A13|     1|       5|                1|   28|     2|   1|
|2008|    4|  1|    2|     29|         1|                  1|                 A16|     1|       6|                1|   33|     2|   1|
|2008|    4|  1|    3|     29|         1|                  2|                  B4|    10|       2|                1|   52|     1|   1|
|2008|    4|  1|    4|     29|         1|                  2|                 B17|     6|       6|                2|   38|     2|   1|
|2008|    4|  1|    5|     29|         1|              

Defining the batches

In [9]:
from pyspark.sql.functions import *

In [10]:
df_month_4 = df1.where(df1.month == "4")
df_month_5 = df1.where(df1.month == "5")
df_month_6 = df1.where(df1.month == "6")
df_month_7 = df1.where(df1.month == "7")
df_month_8 = df1.where(df1.month == "8")

Calculating the sales days for each batch/month

In [11]:
max_day_4 = df_month_4.groupBy('day', 'month')\
    .agg(sum("price").alias("sum_price")) \
    .sort(desc("sum_price")) \

max_day_5 = df_month_5.groupBy('day', 'month')\
    .agg(sum("price").alias("sum_price")) \
    .sort(desc("sum_price")) \

max_day_6 = df_month_6.groupBy('day', 'month')\
    .agg(sum("price").alias("sum_price")) \
    .sort(desc("sum_price")) \

max_day_7 = df_month_7.groupBy('day', 'month')\
    .agg(sum("price").alias("sum_price")) \
    .sort(desc("sum_price")) \

max_day_8 = df_month_8.groupBy('day', 'month')\
    .agg(sum("price").alias("sum_price")) \
    .sort(desc("sum_price")) \


In [12]:
res = max_day_4.union(max_day_5)
result = res.union(max_day_6)
result_t = result.union(max_day_7)
result_table = result_t.union(max_day_8)
result_table.show(100)

+---+-----+---------+
|day|month|sum_price|
+---+-----+---------+
|  2|    4|   150387|
|  1|    4|   139570|
|  3|    4|    81466|
| 10|    4|    80551|
|  7|    4|    80067|
|  4|    4|    78843|
| 18|    4|    78416|
| 14|    4|    77543|
| 29|    4|    77363|
|  8|    4|    74633|
| 16|    4|    74126|
| 17|    4|    71210|
| 11|    4|    70897|
| 21|    4|    69749|
|  5|    4|    66579|
| 19|    4|    65131|
|  9|    4|    64664|
| 23|    4|    64319|
| 22|    4|    64016|
| 28|    4|    63914|
| 15|    4|    61424|
|  6|    4|    57531|
| 12|    4|    56661|
| 20|    4|    55043|
| 24|    4|    53152|
| 30|    4|    52512|
| 25|    4|    47068|
| 27|    4|    45079|
| 13|    4|    42061|
| 26|    4|    36949|
| 14|    5|    69781|
| 29|    5|    66287|
| 16|    5|    65744|
| 19|    5|    65337|
|  7|    5|    64681|
| 21|    5|    63043|
| 25|    5|    62811|
|  5|    5|    62577|
| 12|    5|    60852|
|  8|    5|    60416|
| 20|    5|    58769|
| 28|    5|    55948|
| 27|    5

Continue only if the table schema is setup correctly in BigQuery

In [ ]:
#Define the GCS bucket
gcs_bucket = "geitenemmer"

#table schema needs to be the same in BigQuery

#write to an existing dataset.table
result_table.write \
    .format("bigquery") \
    .option("temporaryGcsBucket", gcs_bucket) \
    .option("checkpointLocation", "gs://geitenemmer/checkpoints") \
    .option("table", "batch_dataset.result_table") \
    .mode("append") \
    .save()

In [ ]:
%%bigquery
SELECT month
FROM batch_dataset.result_table
ORDER BY month DESC
LIMIT 10